# Network automation


## SSH automation with Fabric

This section is adapted from the notes by Jason McVetta from here: https://github.com/jmcvetta/curriculum-advanced-python.git. Licence: CC BY 3.0.

Fabric is a simple but powerful SSH automation tool that is widely used in industry. It streamlines the use of SSH for application deployment or systems administration tasks.

Fabric provides a basic suite of operations for executing local or remote
shell commands (normally or via sudo) and uploading/downloading files,
as well as auxiliary functionality such as prompting the running user
for input, or aborting execution.

You can install fabric with `pip install fabric`

In [68]:
! fab --version

Fabric 2.3.1
Paramiko 2.4.1
Invoke 1.1.0


## Fabric `Connection`

The most basic use of Fabric is to connect to a shell via a `Connection`, execute a command (via the Python [Invoke](http://www.pyinvoke.org/) library) and return the result.

In [6]:
import getpass

In [7]:
username = getpass.getuser()
password = getpass.getpass('Please enter your password: ')

Please enter your password: ········


In [3]:
from fabric import Connection

In [15]:
Connection('localhost', user=username, connect_kwargs={'password': password}).run('uname -s')

Darwin


<Result cmd='uname -s' exited=0>

Note that The response from Fabric by default will echo to the system standard out, as well as returning a `Result` object. Passing the `hide` keyword argument will stop the automatic echo, though the server response can be easily obtained from the result object.

In [18]:
result = Connection('localhost', user=username, connect_kwargs={'password': password}).run('uname -s', hide=True)

In [19]:
result.stdout

'Darwin\n'

### A note on SSH

Fabric uses the underlying Python [Paramiko](http://www.paramiko.org/) library to connect via Secure Shell (SSH) to a given machine. On most Linux / macOS machines this is not too difficult to set up - you will likely have an [OpenSSH](https://www.openssh.com/) service installed and enabled. On a Windows machine this can be a little more tricky as you will likely need Administrator privileges to [install and configure](https://winscp.net/eng/docs/guide_windows_openssh_server) the server.

As such for the purpose of training you have access to the following access details:

| | |
|---|---|
| Username | trainee |
| Password | ****** |
| Host | ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com |

Ask us for the password in your training course.

In [24]:
conn = Connection(
    'ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com',
    user='trainee',
    connect_kwargs={'password': password}
)

In [25]:
conn.run('cat success.txt')

Connection successful


<Result cmd='cat success.txt' exited=0>

### Executing commands over a `Connection`

Once you have a `Connection` set up you can use Fabric to execute commands directly on the server. You've already seen the `run` command which will execute a shell command on the server and return the response:

In [28]:
conn.run('whoami')

trainee


<Result cmd='whoami' exited=0>

Other commands are available:

- `sudo` allowing a user to use `sudo` permissions to execute a command on the server, assuming of course the user has permission
- `cd` changes directory - this can also be used as a context manager
- `put` allows a user to copy a file from the local system to the remote system

In [34]:
conn.put('Data/Auto.csv', remote='Auto.csv')
conn.run('ls')

Auto.csv
success.txt


<Result cmd='ls' exited=0>

#### Exercise: Making your mark on an open server

Make your mark on an open server by leaving a sign if you were there. Connect to the open server above and copy across the file `Data/kilroy_was_here.png` to the server with your own name on the file.

![](extras/kilroy_was_here.png)

Run the `ls` command to guarantee that the file is there based on the `stdout`.

In [40]:
#%load solutions/kilroy.py

You can also copy files across from the server to your local machine with the `get` method:

In [42]:
conn.get('henry.png', 'local_file.png')

In [44]:
import os
os.path.exists('local_file.png')

True

### Writing commands on multiple servers

It's all well and good to connect to different machines individually and complete the same work:

In [45]:
for host in [
        'ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com',
        'ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com']:
    conn = Connection(
        host,
        user='trainee',
        connect_kwargs={'password': password}
    )
    conn.run('uname -s')

Linux
Linux


That works well if you want to execute tasks in order. Instead it's more likely that you'll want to execute the same commands synchronously across different hosts. This is easily using a fabric `Group`. You can subclass the `Group` class yourself, but the easiest way to use groups is to use the built-in `SerialGroup` or the `ThreadingGroup` which execute in serial and parallel respectively.

In [52]:
from fabric import SerialGroup, ThreadingGroup

This is easy if you have a the host strings set up appropriately using fabric configuration (see below), but if you have to pass extra arguments like the username and the password you can make a group from collections:

In [48]:
connections = []
for host in [
        'ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com',
        'ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com']:
    conn = Connection(
        host,
        user='trainee',
        connect_kwargs={'password': password}
    )
    connections.append(conn)

In [54]:
results = ThreadingGroup.from_connections(connections=connections).run('uname -s')

Linux
Linux


The `ResultsGroup` that is returned is an extended dictionary keyed off the original connections, but with two important attributes:

In [59]:
results.succeeded

{<Connection host=ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com user=trainee>: <Result cmd='uname -s' exited=0>}

In [60]:
results.failed

{}

Which split results into successes and fails respectively.

## Configuring fabric hosts

So you don't have to keep configuring long host names, usernames and passwords the best strategy is often to build configuration files to go with Fabric.

By default Fabric will read in your SSH configuration from: `~/.ssh/config`. Note that your SSH config will not support storage of a raw password, so for the example connection we have above you should add the following lines to `~/.ssh/config`:

In [ ]:
Host python_trainee
     HostName ec2-13-211-219-60.ap-southeast-2.compute.amazonaws.com
     Port 22
     User trainee

Then from the command line run

In [ ]:
ssh-copy-id python_trainee

After entering your password your computer's identity will be recorded on the remote machine and you won't need to record a password the next time you log into the machine.

In [76]:
! fab -H python_trainee hostname

ip-172-31-5-84


You can also override configuration in the the Fabric system, based on the [Invoke](http://docs.pyinvoke.org/en/latest/concepts/configuration.html#configuration) library configuration hierarchy:

1. Internal default values
2. Collection-driven configuration
3. System level configuration, stored in `/etc/fabric.yml`
4. User-level configuration file in `~/.fabric.yml`
5. Project-level configuration file in the current project in `fabric.yml`
6. Via Environment variables
7. Runtime configuration file whose path is given to `-f`, e.g. `fab -f /random/path/to/config_file.yaml`.
8. Command-line flags for certain core settings, such as `-e`.
9. Modifications made by user code at runtime.

## `fab` Command Line Interface

Fabric provides a command line utility `fab`, which reads its
configuration from a file named `fabfile.py` in the directory from which it
is run. A typical fabfile contains one or more functions to be executed
on a group of remote hosts.

The following example provides functions to check free disk space and
host type, as well as defining a group of hosts on which to run:

In [80]:
%%writefile fabfile.py
from fabric import task

@task
def hosttype(conn):
    conn.run('uname -s')

@task
def hostname(conn):
    conn.run('hostname')

Overwriting fabfile.py


Fabric can be invoked from the command line with the `fab` command, which by default will look functions decorated with tasks in `fabfile.py`. For example:

In [82]:
! fab --list

Available tasks:

  hostname
  hosttype



Once a task is defined, it may be run on one or more servers, like so:

In [81]:
! fab -H python_trainee hosttype

Linux


### Task arguments

It's often useful to pass runtime parameters into your tasks, just as you might during regular Python programming. Fabric has basic support for this using a shell-compatible notation: `<task name> <arg> --<kwarg>=<value>,...`. It's contrived, but let's extend the above example to say hello to you personally:

In [94]:
%%writefile fabfile.py
from fabric import task

@task
def hello(c, name="world"):
    print("Hello %s!" % name)

Overwriting fabfile.py


By default, calling `fab hello` will still run and print "Hello world!", but you can also pass through command line arguments:

In [98]:
! fab hello --name Henry

Hello Henry!


For now, your argument values will always show up in Python as strings and may require a bit of string manipulation for compound types such as lists. Future versions may add a typecasting system to make this easier.

### Library Usage

In addition to use via the fab tool, Fabric’s components may be imported into other Python code, providing a Pythonic interface to the SSH protocol suite at a higher level than that provided by e.g. the `ssh` library (which Fabric itself uses.)

#### Exercise: Current IP address

Consider the case where we want to collect the IP address from a group of hosts.

1. Write a script that uses the Fabric library to poll a group of hosts for their IP addresses, parses it and displays the IP address to the user:

Use this as a template:

In [1]:
from fabric import task


@task
def get_ip_address(conn):
    result = conn.run('ip addr show', hide=True)

    # write your parsing code here

Test it on the `python_trainee` host above.

## More libraries for network automation

- telnetlib: part of the official Python standard library: https://docs.python.org/3.6/library/telnetlib.html
- Ansible
- Paramiko
- Netmiko
- PySNMP
- Juniper's PyEZ library
- Arista's eAPI
- NAPALM